In [4]:
from get_data import get_dataset

tds, vds = get_dataset()

ModuleNotFoundError: No module named 'get_data'

/home/khj6051/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['audio', 'transcripts'],
    num_rows: 160859
})

In [ ]:
from IPython.display import Audio

for i  in range(100, 200):
    x  = ds[i]["audio"]["array"]
    sr = ds[i]["audio"]["sampling_rate"]
    
    print(ds[i]['transcripts'])
    # display(Audio(x, rate=sr))

In [ ]:
print(ds.cache_files['test'][0]['filename']+"/0004.wav")
import os

os.path.exists(ds.cache_files['test'][0]['filename']+"/0004.wav")

In [ ]:
splits = ds.train_test_split(test_size=0.1, seed=42)  # seed 고정하면 재현성 O

train_ds = splits["train"]
val_ds   = splits["test"]

print(len(train_ds), len(val_ds))

In [ ]:
from datasets import load_dataset

ds = load_dataset("kresnik/zeroth_korean")["train"]
splits = ds.train_test_split(test_size=0.1, seed=42)  # seed 고정하면 재현성 O

train_ds = splits["train"]
val_ds   = splits["test"][:3000]

In [ ]:
from datasets import load_dataset

ds = load_dataset("Junhoee/STT_Korean_Dataset_80000")

data = next(iter(ds['train']))
print(data['audio'])
print(data['transcripts'])

data['audio']['array'].shape

In [ ]:
from datasets import load_dataset
path = "Emilia/KO/*.tar" # Same for Emilia-YODAS; just replace "Emilia/" with "Emilia-YODAS/"
dataset = load_dataset("amphion/Emilia-Dataset", data_files={"ko": path}, split="ko", streaming=False)
splits = dataset.train_test_split(test_size=0.1, seed=42)  # seed 고정하면 재현성 O

train_ds = splits["train"]
val_ds   = splits["test"]


In [ ]:
len(val_ds)

In [ ]:
import time
st = time.time()

audios = []
refs = []
for s in batch_samples:
    audio_np, ref_text = preprocess_sample(s)
    audios.append(audio_np.to(device))
    refs.append(ref_text)


In [ ]:
import time
st = time.time()
results = pipe(
    audios[0],
)
print(time.time() - st)

In [ ]:
with torch.inference_mode():
    for batch_samples in tqdm(chunked(limited_iter, BATCH), desc="Batched ASR"):
        # 전처리
        audios: List[np.ndarray] = []
        refs: List[str] = []
        for s in batch_samples:
            audio_np, ref_text = preprocess_sample(s)
            audios.append(audio_np)
            refs.append(ref_text)

        # 파이프라인에 리스트로 입력 (배치 추론)
        # 긴 파일 대비 안전하게 chunk/stride 설정
        results = pipe(
            audios,
            batch_size=BATCH,
            return_timestamps=False
        )
        # results: List[{'text': ...}, ...]
        hyps = [r["text"] for r in results]

        # 메트릭 누적
        for hyp, ref in zip(hyps, refs):
            wers.append(wer(ref, hyp, unit="word"))
            cers.append(cer(ref, hyp))

        total_count += len(batch_samples)

        # 주기 출력/저장
        if total_count % 500 < BATCH:  # 500단위 근처에서 한 번
            avg_wer = sum(wers) / len(wers) if wers else 0.0
            avg_cer = sum(cers) / len(cers) if cers else 0.0
            line = f"[{total_count}th] - WER: {avg_wer:.6f}, CER: {avg_cer:.6f}"
            print(line)
            with open("wer.txt", "a", encoding="utf-8") as f:
                f.write(line + "\n")

# 최종 기록
final_wer = sum(wers) / len(wers) if wers else 0.0
final_cer = sum(cers) / len(cers) if cers else 0.0
with open("wer.txt", "a", encoding="utf-8") as f:
    f.write(f"[{total_count}th] - WER: {final_wer:.6f}, CER: {final_cer:.6f}\n")

print(f"DONE: N={total_count}, WER={final_wer:.6f}, CER={final_cer:.6f}")


In [36]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import librosa

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 10010.27it/s]
Device set to use cuda:0


In [38]:
import numpy as np
from datasets import load_dataset, Audio
from utils import clean_transcript

ds = load_dataset("idiotDeveloper/koreanTelephone")

ds = ds.cast_column("audio", Audio(decode=True, sampling_rate=16000))

print(len(ds['test']))
arr = np.array(ds['test']["transcripts"])
mask = np.array([not ("((" in t or "))" in t) for t in arr])
ds['test'] = ds['test'].select(np.where(mask)[0])
print(len(ds['test']))

ds = ds["test"].select(range(100)).map(clean_transcript)

443647
418571


Map: 100%|██████████| 100/100 [00:00<00:00, 2081.92 examples/s]


In [ ]:
from tqdm import tqdm
from IPython.display import Audio


for idx, data in tqdm(enumerate(ds)):
    if idx<10:
        continue
        
    audio = torch.tensor(data['audio']['array']).to(device)
    result = pipe(audio)
    display(Audio(audio.cpu().numpy(), rate=16000))
    print(data['transcripts'])
    print(result['text'])

    if idx > 30:
        break